# Jupyter notebook to query the harvested metadata records from the IISG bibliographic materials (biblio)

This notebook makes it possible to get overviews and query the metadata records of the International Institute of Social History (IISG) Bibliographic materials ("Biblio"). It uses as source the file "converted.csv" obtained via metadata harvesting using the scripts in this repository (https://github.com/lilimelgar/iisg-metadata-overviews).  It contains MARC records from the OAIPMH endpoint. 
The file contains one record per row, and each marc property (field and subfield) is in a column.

Note: the data includes only metadata records at the "item" level.

Created by Liliana Melgar (April, 2024).

# A. Set up

## A1. Import the required python libraries 
*(nothing to change)*

In [1]:
import pandas as pd
import numpy as np
import csv
import re

from IPython.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# to add timestamp to file names
import time
# import os.path to add paths to files
import os

## A2. Set the path to the csv file 
*nothing to change if you cloned the repository. If you downloaded the file only ("biblio_as_csv.gzip"), then set here the path to where you have downloaded the file*

In [2]:
# path to where the transformed csv is located
data_directory = os.path.abspath(os.path.join('..', 'data'))
data_converted = os.path.join(data_directory, 'converted') #path to the repository folder where the csv file is located, if you have not cloned the repository, change the path here
data_downloads = os.path.join(data_directory, 'downloads') #path to the folder where the reports will be downloaded

## A3. Read the csv file as a pandas dataframe
*nothing to change here, just be patient, IT TAKES LONG TO LOAD (around started at 19.00h and finished sometime before 20:48h same day)*

In [3]:
# read csv as dataframe
biblio_df_v0 = pd.read_csv(f'{data_converted}/biblio_as_csv.gzip', sep="\t", compression='gzip', low_memory=False)
# low_memory=False was set after this warning message: "/var/folders/3y/xbjxw0b94jxg6x2bcbyjsmmcgvnf7q/T/ipykernel_987/2912965462.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False."

# B. First overview and data preparation

## B1. First overview: all fields and data types
Execute the cell and view the general information of the data, which includes the Columns (marc properties with subfields), the Non-Null Count (i.e., how many cells have values; for example: if a cell says "1 non-null" it means that only one row has a value); and the Data type (object (i.e., a string or a combination of data types), a float or an integer).
- Keep in mind that the MARC labels have 3 characters, and that the fourth character can be an indicator or a subfield. For example: 1000 is Marc label 100 with indicator 0. And 100a is Marc label 100 with subfield a.

In [4]:
biblio_df_v0.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204585 entries, 0 to 1204584
Data columns (total 1032 columns):
 #     Column  Non-Null Count    Dtype  
---    ------  --------------    -----  
 0     001     1204585 non-null  int64  
 1     003     1204585 non-null  object 
 2     005     1194633 non-null  float64
 3     006     46 non-null       object 
 4     007     191 non-null      object 
 5     008     1204585 non-null  object 
 6     010a    5443 non-null     object 
 7     010z    23 non-null       object 
 8     012a    4 non-null        object 
 9     012b    1 non-null        float64
 10    012i    1 non-null        float64
 11    012z    1 non-null        object 
 12    0152    2091 non-null     object 
 13    015a    2167 non-null     object 
 14    015z    42 non-null       object 
 15    0162    1975 non-null     object 
 16    016a    2010 non-null     object 
 17    016z    1 non-null        float64
 18    017a    32 non-null       object 
 19    017b    22 non-nu

## B2. Optional (documentation)
Ideally, each field above would have a definition explaining what it means and what kind of values does it contain (in relation to the conventions for creating IISG metadata). That documentation can exist somewhere else (e.g., on Confluence), but this could be a place to start updating or writing those definitions since here one can see the data that they contain in detail.

## B3. Prepare the data for search
Because we know that the data doesn't have proper numerical values to be computed, we rather convert all values to strings in order to facilitate querying. This also includes filling in empty values with a standard string: "null"
*(nothing to change here)*

In [5]:
# convert datatypes and fill in empty values
df_columns = biblio_df_v0.columns
for column in df_columns:
    dataType = biblio_df_v0.dtypes[column]
    if dataType == np.float64:
        biblio_df_v0[column] = biblio_df_v0[column].fillna('null')
        biblio_df_v0[column] = biblio_df_v0[column].astype(str)
    if dataType == np.int_:
        biblio_df_v0[column] = biblio_df_v0[column].fillna('null')
        biblio_df_v0[column] = biblio_df_v0[column].astype(str)
    if dataType == object:
        biblio_df_v0[column] = biblio_df_v0[column].fillna('null')
        biblio_df_v0[column] = biblio_df_v0[column].astype(str)

In [6]:
# create a copy
biblio_df = biblio_df_v0.copy()

In [7]:
# # save the csv
# biblio_df.to_csv('biblio_all.csv.gz', index=False, compression='gzip')

In [8]:
# Check again the general information of the data after having filled in the emtpy values and converted the data types
biblio_df.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204585 entries, 0 to 1204584
Data columns (total 1032 columns):
 #     Column  Non-Null Count    Dtype 
---    ------  --------------    ----- 
 0     001     1204585 non-null  object
 1     003     1204585 non-null  object
 2     005     1204585 non-null  object
 3     006     1204585 non-null  object
 4     007     1204585 non-null  object
 5     008     1204585 non-null  object
 6     010a    1204585 non-null  object
 7     010z    1204585 non-null  object
 8     012a    1204585 non-null  object
 9     012b    1204585 non-null  object
 10    012i    1204585 non-null  object
 11    012z    1204585 non-null  object
 12    0152    1204585 non-null  object
 13    015a    1204585 non-null  object
 14    015z    1204585 non-null  object
 15    0162    1204585 non-null  object
 16    016a    1204585 non-null  object
 17    016z    1204585 non-null  object
 18    017a    1204585 non-null  object
 19    017b    1204585 non-null  object
 20  

# C. Get a glimpse of the data

## C1. First rows
Here you can see a sample of the records, one per line. You can change the value "10" to any other desired size for your sample, preferably not too big. You can also use "tail" instead of "head" to see the records in the last rows.
- Keep in mind to scroll horizontally and vertically to see the entire record.
- NaN means that the cell is empty.
- Arbitrarily, some cells above, we decided that the omega "Ω" would be the separator for multi-value cells.

In [9]:
biblio_df.head(10)

001       003               005   006   007  \
0  1000000  NL-AMISG  19971030111032.0  null  null   
1  1000001  NL-AMISG  19980525124255.0  null  null   
2  1000002  NL-AMISG  19981207144010.0  null  null   
3  1000003  NL-AMISG  19970409135516.0  null  null   
4  1000004  NL-AMISG  19970409135738.0  null  null   
5  1000005  NL-AMISG  19971030111542.0  null  null   
6  1000006  NL-AMISG  19980525124520.0  null  null   
7  1000007  NL-AMISG  19981207144812.0  null  null   
8  1000008  NL-AMISG  19980525124959.0  null  null   
9  1000009  NL-AMISG  19970409140006.0  null  null   

                                        008  010a  010z  012a  012b  012i  \
0  199710suuuuuuuuru                  rus d  null  null  null  null  null   
1  199805suuuuuuuune                  dut d  null  null  null  null  null   
2  199812s1968uuuuxxk                 eng d  null  null  null  null  null   
3  199704suuuuuuuuxxk                 eng d  null  null  null  null  null   
4  199704suuuuuuuuxxu                 eng d  null  null  null  null  null   
5  199710s1994uuuuru                  rus d  null  null  null  null  null   
6  199805s1898uuuuxxk                 eng d  null  null  null  null  null   
7  199812suuuuuuuufr                  fre d  null  null  null  null  null   
8  199805suuuuuuuune                  dut d  null  null  null  null  null   
9  199704suuuuuuuuxxu                 eng d  null  null  null  null  null   

   012z  0152  015a  015z  0162  016a  016z  017a  017b  019a  0209  \
0  null  null  null  null  null  null  null  null  null  null  null   
1  null  null  null  null  null  null  null  null  null  null  null   
2  null  null  null  null  null  null  null  null  null  null  null   
3  null  null  null  null  null  null  null  null  null  null  null   
4  null  null  null  null  null  null  null  null  null  null  null   
5  null  null  null  null  null  null  null  null  null  null  null   
6  null  null  null  null  null  null  null  null  null  null  null   
7  null  null  null  null  null  null  null  null  null  null  null   
8  null  null  null  null  null  null  null  null  null  null  null   
9  null  null  null  null  null  null  null  null  null  null  null   

         020a  020c  020q  020z  0222  022a  022e  022l  022z  0242  024a  \
0        null  null  null  null  null  null  null  null  null  null  null   
1        null  null  null  null  null  null  null  null  null  null  null   
2  0434922536  null  null  null  null  null  null  null  null  null  null   
3  0521087430  null  null  null  null  null  null  null  null  null  null   
4        null  null  null  null  null  null  null  null  null  null  null   
5  5842800421  null  null  null  null  null  null  null  null  null  null   
6        null  null  null  null  null  null  null  null  null  null  null   
7        null  null  null  null  null  null  null  null  null  null  null   
8        null  null  null  null  null  null  null  null  null  null  null   
9        null  null  null  null  null  null  null  null  null  null  null   

   024q  025a  0262  0265  026a  026b  026c  026d  026e  027a  027q  028a  \
0  null  null  null  null  null  null  null  null  null  null  null  null   
1  null  null  null  null  null  null  null  null  null  null  null  null   
2  null  null  null  null  null  null  null  null  null  null  null  null   
3  null  null  null  null  null  null  null  null  null  null  null  null   
4  null  null  null  null  null  null  null  null  null  null  null  null   
5  null  null  null  null  null  null  null  null  null  null  null  null   
6  null  null  null  null  null  null  null  null  null  null  null  null   
7  null  null  null  null  null  null  null  null  null  null  null  null   
8  null  null  null  null  null  null  null  null  null  null  null  null   
9  null  null  null  null  null  null  null  null  null  null  null  null   

   029a  029b  02ea  02ga  02sa  030a  033a  033b  034a  0359  \
0  null  null  null  

## C2. Size (shape) of the data
Here you can see how many rows (first value) and how many columns (second value) are in the data.

In [10]:
biblio_df.shape

(1204585, 1032)

## C3. Unique values
Here you can see a general description of the data, including how many unique values are per column.

In [11]:
# describe the dataframe
biblio_df.describe()

,001,003,005,006,007,008,010a,010z,012a,012b,012i,012z,0152,015a,015z,0162,016a,016z,017a,017b,019a,0209,020a,020c,020q,020z,0222,022a,022e,022l,022z,0242,024a,024q,025a,0262,0265,026a,026b,026c,026d,026e,027a,027q,028a,029a,029b,02ea,02ga,02sa,030a,033a,033b,034a,0359,035a,035f,035z,036a,037a,037b,037c,037n,040a,040b,040c,040d,040e,0412,041a,041b,041c,041d,041e,041f,041h,041k,041m,041n,042a,043a,043c,044a,044b,044c,045a,045b,046a,047a,048a,049a,04ka,04ra,050a,050b,050c,051a,051b,051c,052a,055a,055b,060a,060b,066c,070a,070b,0722,072a,072x,078a,0802,080a,080q,0822,0828,082a,082b,082q,0832,0838,083a,083q,0840,0842,0849,084a,084b,084q,0858,085b,085s,085z,0862,086a,088a,090a,090b,0922,092a,092b,096a,096b,097a,097b,099a,100,1000,1001,1003,1004,1006,100D,100a,100b,100c,100d,100e,100p,100q,110,1100,1104,1106,110a,110b,110c,110d,110e,110g,110n,1110,1114,1116,111a,111b,111c,111d,111e,111j,111n,1300,1306,130a,130f,130k,130l,130p,199a,199b,199c,202a,210a,210b,222a,222b,224a,2406,240a,240f,240h,240k,240l,240y,241a,242,2421,2426,242a,242b,242c,242y,244a,245,2450,2456,2458,245B,245C,245D,245H,245P,245U,245a,245b,245c,245d,245e,245f,245h,245j,245k,245n,245p,245v,245z,246,2461,2466,246a,246b,246c,246d,246i,246k,246n,250,2506,250a,250b,250c,250g,255a,257a,260,2601,2602,2606,260B,260I,260L,260M,260S,260a,260b,260c,260d,260e,260f,260g,260k,260l,260q,260s,260x,263a,264,2641,2644,2646,264a,264b,264c,264d,264e,264f,264g,264x,264©,265c,269a,270a,270b,270c,294c,300,3002,3003,3004,3006,300a,300b,300c,300d,300e,300g,300v,310a,310b,321a,321b,3360,3362,3363,336a,336b,336c,336d,336q,3370,3372,3373,337a,337b,337c,3380,3382,3383,338a,338b,338c,338e,3402,340a,340m,340n,3442,344b,344c,344i,347a,347b,350a,362a,362z,3662,366b,366c,366f,366j,366k,366m,3682,368a,368b,3802,380a,3852,385a,3862,386a,386i,386m,386n,3882,388a,400a,440a,440p,440v,440x,446a,452m,480a,480v,490,4900,4906,490a,490b,490d,490v,490x,500,5005,5006,5008,500A,500a,500b,500c,500d,500e,500f,500g,500r,500s,500t,500v,501a,502a,502b,502c,502d,502g,5046,504a,5056,505A,505a,505b,505g,505r,505s,505t,505v,5062,5063,506U,506a,506b,506d,506f,506u,508a,510a,510b,510c,510x,511a,513a,515a,518a,5203,5206,520a,520c,520u,521a,525a,530a,532a,533a,533b,533c,533d,533e,533f,533m,533n,534a,534c,534f,534n,534p,534t,535a,538a,539a,539b,539d,539e,5402,5405,540a,540b,540c,540f,540u,5410,5413,541a,541b,541c,541d,541e,541h,542a,542b,542c,542d,542f,542g,542i,542j,542m,545a,5466,546a,546b,550a,555a,555u,560a,561a,5633,5635,563a,580a,5832,5835,5838,583a,583c,583d,583f,583i,583l,583u,583z,585a,586a,588a,590a,600,6000,6001,6002,6006,600A,600a,600b,600c,600d,600e,600f,600l,600n,600q,600t,600v,600x,600y,600z,6100,6101,6102,6106,6109,610a,610b,610c,610d,610e,610f,610g,610n,610t,610v,610x,610y,610z,6110,6112,6116,611a,611b,611c,611d,611e,611f,611k,611n,611q,611t,611x,611z,630,6300,6302,6306,630A,630a,630b,630c,630d,630e,630f,630g,630p,630v,630x,630z,6470,6471,6472,647a,647c,647d,6480,6482,6486,648a,648e,650,6500,6501,6502,6504,6506,6508,650a,650d,650g,650l,650v,650x,650y,650z,6510,6511,6512,6516,651a,651b,651g,651v,651x,651y,651z,6536,653a,653d,654a,6550,6551,6552,6555,6556,655a,655b,655c,655v,655x,655y,655z,690a,690b,690c,691a,691b,692a,692b,693a,693b,6946,694a,694b,695a,695c,695d,695e,695f,695g,695h,695w,695z,696a,696z,697a,697b,700,7000,7001,7004,7005,7006,700A,700D,700E,700a,700b,700c,700d,700e,700f,700g,700i,700k,700l,700n,700p,700q,700r,700s,700t,700x,700z,710,7100,7101,7102,7104,7105,7106,7109,710a,710b,710c,710d,710e,710f,710g,710k,710m,710n,710o,710r,710s,710t,710v,710x,710z,711(,7110,7114,7116,711a,711b,711c,711d,711e,711f,711g,711j,711k,711l,711n,711q,711t,711x,711z,720a,720e,7300,7305,7306,730a,730g,730i,730n,740,7406,740a,740b,740e,740n,745a,751a,751e,7522,7524,752a,752b,752d,752e,752f,752g,7581,7584,758a,758i,760d,760g,760k,760q,760t,760w,760x,7656,765a,765b,765d,765i,765t,765w,765z,772a,772b,772d,772h,772i,772t,772w,772x,7737,773a,773g,773i,773q,773t,773w,773x,774a,774g,774t,774w,775a,775d,775i,775n,775t,775w,775z,77

# D. Check the values in one column (marc property)
At this point you may be curious to know which values are in one column. For example, 100e has only 3 unique values, which are those?
- You can change the field inside the quotation marcs for any other field of interest.

In [ ]:
# TEST (see one record)
# check if a string value exists in a column (the string is exactly the same)
# test_exact = biblio_df[biblio_df['651a'] == '1362253']
test_exact = biblio_df[biblio_df['651a'] == 'Srebrenica (Yugoslavia)']
test_exact

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
name_file = 'biblio_651a_Srebrenica'

test_exact.to_excel(f'{data_downloads}/{name_file}.xlsx')

In [ ]:
# biblio_df['100a'].unique().tolist()

## D1. Create a subset with certain column(s)/field(s)
At this point you may have thought that you could perhaps correct some of the records which contain an inconsistent value. For example, in the first version of this data, if you queried above for "biblio_df['100e'].unique()" you may have obtained certain values. You may decide that you want to change one or some of them into another value. But for this, you need the TCN (record Id) numbers. The command below facilitates creating a subset with the TCN and the field of interest.


In [ ]:
# create subset with record Id and record of interest, here enter the name of the field(s) that you are interested in separated by commas, each field has to be within single quotation marks, e.g., biblio_df[['001','100e', '110e']]
# field_subset_df = biblio_df[['001','090a','901a','245a','245b','260a','852p','852j','866a','902a','leader']] #--> For LA periodicals
field_subset_df = biblio_df[['001','245a','245b','6510','651a','695g','leader']] #--> For geographic terms exploration
# field_subset_df

In [ ]:
# check again the number of unique values in your subset
field_subset_df.describe()

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
# name_file = 'biblio_author_person_field_100a' #--> authors test
name_file = 'biblio_geo_651a' #--> geoterms

# field_subset_df.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
field_subset_df.to_csv(f'{data_downloads}/{name_file}.csv', index=False) # if too big, use compression='gzip'

## D2. Create a subset of records with a certain value in a given column
You may also want to create a list of the records with a certain value in a given column, for example, for field 100e you got these unique values: ['creator.', 'null', 'creator']. You may want to get only the list of records that have "creator."

In [ ]:
# when the file above is too big, it's useful sometimes to download it and upload it here again
path = '/Users/lilianam/workspace/iisg-metadata-overviews/biblio/data'
field_subset_df = pd.read_csv(f'{path}/biblio_titles.csv.gz', sep=",", compression='gzip', low_memory=False)

In [ ]:
field_subset_df.head(5)

In [ ]:
# check if a string value exists in a column (the string is exactly the same)
query_value_exact = field_subset_df[field_subset_df['100a'] == 'Hajnal, Henri.']
query_value_exact

In [ ]:
# check if a string value exists in a column (the string is approximately the same)
# you may want to find the records that have either "creator." (with dot) or "creator" without dot, but not the null values
# here it's possible to use regular expressions

query_value_aprox = field_subset_df[field_subset_df['852j'].str.contains("ZDF|ZF|ZDK|ZO|XZK|ZDO|ZK", case=True, regex=True)]

In [ ]:
query_value_aprox.head(5)

In [ ]:
# get some idea of how many rows are in this set
query_value_aprox.info(verbose = True, show_counts = True)

In [ ]:
# check again the number of unique values in your subset
query_value_aprox.describe()

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
# name_file = 'biblio_author_person_field_100a_henri'
name_file = 'biblio_to_map_la_periodicals_852j'

query_value_aprox.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# query_value_aprox.to_csv()

# E. Create subsets using inverse query
You may need to create a report with all the records that do not contain a certain value. For example, because we used "null" to fill in all empty values, one could create a list with all the records that have a value in a certain column.

In [ ]:
# create a slice with the records that have non-null values in the column of interest
# Note: if you want to query the subset instead of the whole data, then replace "biblio_df" with "field_subset_df" and run the cell again

query_inverse = biblio_df[~biblio_df['100a'].str.contains("null", case=False, regex=True)]

query_inverse.head(10)

In [ ]:
# get some info about the subset you got as a result of the query:
query_inverse.info(verbose=True, show_counts = True)

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
name_file = 'biblio_author_person_field_100a_notEmpty'

query_inverse.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# query_inverse.to_csv()

# F. Query for a specific record
You may want to see the details of a specific record, this can be done in two ways:

In [ ]:
# 1. by using the index position. Example: This item: ToDo has index position 0. 
# This position can be seen in the left corner of the entire table (cell above in Section5: biblio_df.head(10))
# We will query it using the entire version of the data, not the subset

# show record vertically using index position
query_recordIndex = biblio_df.iloc[0]
query_recordIndex

In [ ]:
# 2. By using the record Id using the Marc field 001
query_recordId = biblio_df[biblio_df['001'] == '8']
query_recordId